In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
import math

In [2]:
datapath = "/Users/aavattikutis/Documents/epidemicmodel/cccruns/fits/run2/tables/"
models = ["fulllinearmodel_fit_table.csv","reducedlinearmodelNegBinom_fit_table.csv",
          "reducedlinearmodelq0_fit_table.csv","reducedlinearmodelq0ctime_fit_table.csv",
         "nonlinearmodelq0ctime_fit_table.csv"]
# model_hash = {}
# k = -1
# for model in models:
#     k += 1
#     model_hash[model] = string.ascii_uppercase[k]

# df = pd.DataFrame.from_dict(model_hash, orient='index')
# df.to_csv('../postmodel_derivatives/model_hash.csv', header=False)

In [3]:
rois = []
for model in models:
    df = pd.read_csv(datapath + model) #get rois in all tables (some may have failed)
    rois += list(df.roi.unique())

    
rois = list(set(rois))

#get inferred
theta = df.columns[2:] 
ntheta = len(theta)



In [4]:
#get rois

roi_us = np.sort([i for i in rois if i[:2]=='US'])[::-1]
roi_other = np.sort([i for i in rois if i[:2]!='US'])[::-1]
rois = list(roi_us) + list(roi_other)



In [25]:
def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    return
             

def plot_table_data(roi,col,ax,pos_):
    try:
        for j, model in enumerate(models):
            df = pd.read_csv(datapath + model)
            boxes = [
                {
                'x': i,
                'label' : roi+'_'+model.split('_fit_table.csv')[0],
                'whislo': df.loc[(df.roi==roi)&(df['quantile']==0.025), col].values[0],    # Bottom whisker position
                'q1'    : df.loc[(df.roi==roi)&(df['quantile']==0.25), col].values[0],    # First quartile (25th percentile)
                'med'   : df.loc[(df.roi==roi)&(df['quantile']==0.5), col].values[0],    # Median         (50th percentile)
                'q3'    : df.loc[(df.roi==roi)&(df['quantile']==0.75), col].values[0],     # Third quartile (75th percentile)
                'whishi': df.loc[(df.roi==roi)&(df['quantile']==0.975), col].values[0],    # Top whisker position
                'fliers': []        # Outliers
                }
            ]
            ax.bxp(boxes, positions=[pos_[j]], showfliers=False, vert=False, patch_artist=True,
                   boxprops=dict(facecolor='orange',edgecolor='black'))
#             ax.text(300,pos_[j],roi)
#         ax.set_title(model)
        simpleaxis(ax)
    except: #if roi not in table
        print()
    return
            


pos_ = np.arange(1,len(rois)+1)
dpos = np.linspace(-0.25,0.25,5)

theta_ = ['ll_','waic','loo']
k = 0
for theta in ['ll_']:#theta_[:1]:
    k += 1
    f = plt.figure(k, figsize = (100,200))   
    h = f.gca()
    for i in range(len(rois)):
        pos2_ = [pos_[i]+j for j in dpos]
        plot_table_data(rois[i],theta,h,pos2_)
    simpleaxis(h)
    plt.title(theta, fontsize=24)
    plt.xlim((-2000,0))
    plt.gca().yaxis.grid(True,alpha=0.5)

plt.savefig('modelcomp_'+theta+'_all.png')